In [1]:
using CUDA
using TypedTables
using DataFrames
using Statistics
using CSV

In [4]:
# CUDA

function operation_gpu(A, skalar)
    C = CUDA.zeros(Float32, size(A))
    CUDA.@sync C = A * skalar
    return C
end

function simulate_gpu(A_normal, skalar_normal)
    A = CUDA.convert(CuArray{Float32}, A_normal)
    cuda_start = CUDA.@elapsed result_gpu = operation_gpu(A, skalar_normal)
    return cuda_start
end

simulate_gpu (generic function with 1 method)

In [5]:
# CPU

function operation_cpu(A, skalar)
    C = zeros(Float32, size(A))
    C = A * skalar
    return C
end

function simulate_cpu(A_normal, skalar)
    normal_start = @elapsed result_cpu = operation_cpu(A_normal, skalar)
    return normal_start
end

simulate_cpu (generic function with 1 method)

In [6]:
function simulate(N::Int64, n_loop::Int64)
    println("Total size: $(N*N)")
    results_gpu = []
    results_cpu = []
    is_better_gpu = []
    for i in 1:n_loop
        A_normal = rand(Float32, N, N)
        scalar_normal = rand(Float32)  # Generate a random scalar
        benchmark_gpu = simulate_gpu(A_normal, scalar_normal)
        benchmark_cpu = simulate_cpu(A_normal, scalar_normal)
        push!(results_gpu, benchmark_gpu)
        push!(results_cpu, benchmark_cpu)
        push!(is_better_gpu, benchmark_gpu < benchmark_cpu)
    end
    println("Mean CPU: $(mean(results_cpu))")
    println("Mean GPU: $(mean(results_gpu))")
    return DataFrame(CPU=results_cpu, GPU=results_gpu, Better_GPU=is_better_gpu)
end

simulate (generic function with 1 method)

In [15]:
df = simulate(10, 10)
CSV.write("./1.csv", df)
df

Total size: 100
Mean CPU: 1.0655e-6
Mean GPU: 3.5536003e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,1.121e-6,6.9632e-5,false
2,8.2e-7,2.56e-5,false
3,1.39e-6,3.584e-5,false
4,8.56e-7,5.2192e-5,false
5,4.79e-7,2.8672e-5,false
6,6.49e-7,2.4576e-5,false
7,7.32e-7,2.7712e-5,false
8,8.23e-7,2.7648e-5,false
9,6.23e-7,2.7648e-5,false


In [20]:
df = simulate(50, 10)
CSV.write("./2.csv", df)
df

Total size: 2500
Mean CPU: 1.5514900000000003e-5
Mean GPU: 7.137279e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,1.2977e-5,0.000120832,false
2,8.575e-6,0.000100352,false
3,1.4858e-5,6.0416e-5,false
4,1.5551e-5,7.7824e-5,false
5,1.8005e-5,5.632e-5,false
6,1.7396e-5,5.8368e-5,false
7,1.609e-5,5.8368e-5,false
8,1.7485e-5,5.8368e-5,false
9,1.8119e-5,5.632e-5,false


In [22]:
df = simulate(100, 10)
CSV.write("./3.csv", df)
df

Total size: 10000
Mean CPU: 3.4273700000000004e-5
Mean GPU: 3.17632e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,8.834e-6,4.928e-5,false
2,4.2494e-5,3.6864e-5,true
3,4.3987e-5,3.1744e-5,true
4,4.4164e-5,3.072e-5,true
5,5.0729e-5,2.4576e-5,true
6,4.2094e-5,2.4576e-5,true
7,5.0341e-5,2.4576e-5,true
8,4.7522e-5,3.0688e-5,true
9,6.69e-6,3.584e-5,false


In [26]:
df = simulate(500, 10)
CSV.write("./4.csv", df)
df

Total size: 250000
Mean CPU: 0.0009130456999999999
Mean GPU: 0.00014078079


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.000234481,9.12e-5,true
2,0.000257041,0.000125728,true
3,0.00123921,9.4112e-5,true
4,0.00104755,0.000105344,true
5,0.0012209,0.000110432,true
6,0.000925553,0.00015504,true
7,0.00194809,0.00042704,true
8,0.00131759,0.000107328,true
9,0.00070844,9.8016e-5,true


In [27]:
df = simulate(1000, 10)
CSV.write("./5.csv", df)
df

Total size: 1000000
Mean CPU: 0.0029308645
Mean GPU: 0.0004561088


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.000919153,0.00037072,true
2,0.00282876,0.000346336,true
3,0.00117688,0.000381376,true
4,0.00157219,0.000371776,true
5,0.00412109,0.00082576,true
6,0.00275677,0.000336896,true
7,0.0019523,0.000399744,true
8,0.00487231,0.000497536,true
9,0.00732165,0.000441408,true


In [28]:
df = simulate(5000, 10)
CSV.write("./6.csv", df)
df

Total size: 25000000
Mean CPU: 0.28625631339999996
Mean GPU: 0.007873632


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.241146,0.0080023,true
2,0.568156,0.00790691,true
3,0.119355,0.00776819,true
4,0.177667,0.00779773,true
5,0.826951,0.00784544,true
6,0.0822795,0.00783296,true
7,0.116054,0.00816643,true
8,0.0727916,0.00792067,true
9,0.558327,0.0077809,true


In [29]:
df = simulate(10000, 10)
CSV.write("./7.csv", df)
df

Total size: 100000000
Mean CPU: 0.4188525466
Mean GPU: 0.031236375


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.384317,0.0328959,true
2,0.362538,0.0312188,true
3,0.410932,0.0308902,true
4,0.345522,0.0309139,true
5,0.457299,0.0309746,true
6,0.354571,0.0313863,true
7,0.615672,0.0309407,true
8,0.380106,0.0311961,true
9,0.525034,0.031065,true
